In [1]:
# Import-import
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

import tensorflow as tf
import tensorflow_text as text
from official.nlp import optimization

import numpy as np

tf.get_logger().setLevel('ERROR')

2024-12-06 13:18:22.384509: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733465902.460226  166880 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733465902.481688  166880 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-06 13:18:22.652673: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
@tf.keras.utils.register_keras_serializable('capstone')
class MatthewsCorrelationCoefficient(tf.keras.metrics.Metric):
    def __init__(self, num_classes, dtype=None, name="matthews_correlation_coefficient"):
        super(MatthewsCorrelationCoefficient, self).__init__(name=name)
        self.num_classes = num_classes
        self.c = self.add_weight(name="total_correct_predicted", initializer="zeros", dtype=tf.float32)
        self.s = self.add_weight(name="total_samples", initializer="zeros", dtype=tf.float32)
        self.t = self.add_weight(name="num_true", shape=(num_classes,), initializer="zeros", dtype=tf.float32)
        self.p = self.add_weight(name="num_pred", shape=(num_classes,), initializer="zeros", dtype=tf.float32)

    def update_state(self, y_true, y_pred, sample_weight=None):
        # Convert one-hot labels to integer labels
        if len(y_true.shape) > 1:
            y_true = tf.argmax(y_true, axis=-1)
        if len(y_pred.shape) > 1:
            y_pred = tf.argmax(y_pred, axis=-1)
        
        # Total number of samples
        new_s = tf.cast(tf.size(y_true), tf.float32)
        # Total number of correctly predicted labels
        new_c = tf.reduce_sum(tf.cast(tf.equal(y_true, y_pred), tf.float32))
        
        # Update state
        self.s.assign(tf.add(self.s, new_s))
        self.c.assign(tf.add(self.c, new_c))

        for k in range(self.num_classes):
            k = tf.cast(k, y_true.dtype)
            tk = tf.reduce_sum(tf.cast(tf.equal(y_true, k), tf.float32))
            pk = tf.reduce_sum(tf.cast(tf.equal(y_pred, k), tf.float32))
            self.t[k].assign(tf.add(self.t[k], tk))
            self.p[k].assign(tf.add(self.p[k], pk))
    
    def result(self):
        num = self.c * self.s - tf.reduce_sum(self.t * self.p)
        denom_t = tf.reduce_sum(self.t * self.t)
        denom_p = tf.reduce_sum(self.p * self.p)
        denom = tf.sqrt((self.s**2 - denom_t) * (self.s**2 - denom_p))
        
        mcc = tf.divide(num, denom + 1e-6)
        return mcc
    
    def reset_state(self):
        self.c.assign(0)
        self.s.assign(0)
        self.t.assign(tf.zeros_like(self.t))
        self.p.assign(tf.zeros_like(self.p))
    
    def get_config(self):
        config = super(MatthewsCorrelationCoefficient, self).get_config()
        config.update({
            "num_classes": self.num_classes
            })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [3]:
@tf.keras.utils.register_keras_serializable('capstone')
class Classifier(tf.keras.Model):
  def __init__(self, tfhub_handle_encoder, num_classes, dropout_rate, l2):
    super(Classifier, self).__init__(name="prediction")
    self.num_classes = num_classes
    self.dropout_rate = dropout_rate
    self.l2 = l2
    self.tfhub_handle_encoder = tfhub_handle_encoder
    self.encoder = hub.KerasLayer(self.tfhub_handle_encoder, trainable=True)
    self.dropout = tf.keras.layers.Dropout(dropout_rate)
    self.dense = tf.keras.layers.Dense(num_classes, kernel_regularizer=tf.keras.regularizers.l2(l2))

  def call(self, preprocessed_text):
    encoder_outputs = self.encoder(preprocessed_text)
    pooled_output = encoder_outputs["pooled_output"]
    x = self.dropout(pooled_output)
    x = self.dense(x)
    return x
  
  def get_config(self):
    config = super(Classifier, self).get_config()
    config.update({
        "num_classes": self.num_classes,
        "dropout_rate": self.dropout_rate,
        "l2": self.l2,
        "tfhub_handle_encoder": self.tfhub_handle_encoder
    })
    return config
  
  @classmethod
  def from_config(cls, config):
    return cls(**config)

In [ ]:
# initialize model
model = tf.keras.models.load_model('../my_models/checkpoint_03.tf', custom_objects={"Classifier": Classifier,
                                                                                 "MatthewsCorrelationCoefficient": MatthewsCorrelationCoefficient,
                                                                                 "AdamWeightDecay": optimization.AdamWeightDecay,
                                                                                 "WarmUp": optimization.WarmUp
                                                                                 })
# model = tf.saved_model.load("my_models/model_bert_en_uncased_L-12_H-768_A-12")
classes = np.array(['Anxiety', 'Bipolar', 'Depression', 'Normal', 'Personality disorder', 'Stress', 'Suicidal'])

I0000 00:00:1733455540.887803  134376 gpu_process_state.cc:201] Using CUDA malloc Async allocator for GPU: 0
I0000 00:00:1733455540.889378  134376 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1753 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Ti Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


In [ ]:
text = tf.constant(["tomorrow danish will give some presentation, he feel so nervous, what if he gets diarrhea"])
predictions = model.predict(text)

# predictions will be a numpy array of shape (1, num_classes) with probabilities for each class
print(classes)
print(tf.nn.softmax(predictions))

# To get the predicted class index
predicted_class_index = tf.argmax(predictions, axis=1).numpy()[0]
print(f"Predicted class: {classes[predicted_class_index]}")

ValueError: Could not find matching concrete function to call loaded from the SavedModel. Got:
  Positional arguments (1 total):
    * <tf.Tensor 'preprocessed_text:0' shape=(1,) dtype=string>
  Keyword arguments: {'training': False}

 Expected these arguments to match one of the following 2 option(s):

Option 1:
  Positional arguments (1 total):
    * {'input_mask': TensorSpec(shape=(None, 256), dtype=tf.int32, name='input_mask'),
 'input_type_ids': TensorSpec(shape=(None, 256), dtype=tf.int32, name='input_type_ids'),
 'input_word_ids': TensorSpec(shape=(None, 256), dtype=tf.int32, name='input_word_ids')}
  Keyword arguments: {'training': True}

Option 2:
  Positional arguments (1 total):
    * {'input_mask': TensorSpec(shape=(None, 256), dtype=tf.int32, name='input_mask'),
 'input_type_ids': TensorSpec(shape=(None, 256), dtype=tf.int32, name='input_type_ids'),
 'input_word_ids': TensorSpec(shape=(None, 256), dtype=tf.int32, name='input_word_ids')}
  Keyword arguments: {'training': Fa